In [167]:
import pandas as pd
import numpy as np
from functools import reduce
import os
import matplotlib.backends 

import matplotlib.pyplot as plt
import seaborn as sns


AttributeError: module 'matplotlib' has no attribute 'rcParams'

In [187]:
refs        ='/home/alva/Documents/lncRNAs_project_2020/Analysis_2021/Tcga_R/GDCdata/TCGA-BRCA/harmonized/Transcriptome_Profiling/Gene_Expression_Quantification/r_data/reference/Homo_sapiens.GRCh38.97.chr.txt'
ref         = pd.read_csv(refs, sep="\t", header=0,skipinitialspace=True)
lncRNAs_ref = ref.query('biotype == "lncRNA"')

In [ ]:
# gencode except for the nagative ones 12869
prediction_triplexfpp_genocode_all = pd.read_csv('/home/alva/Documents/lncRNAs_project_2020/scripts_2021/TriplexFPP/output/triplexgencode.v36.lncRNA_transcripts_results.tsv',header=None,sep='\t',names=['headers','score','triplex_helices'])
prediction_triplexfpp_genocode_all = prediction_triplexfpp_genocode_all.join(prediction_triplexfpp_genocode_all['headers'].apply(lambda x: pd.Series(str(x).split('|'))).rename(columns=lambda x: f"string_{x+1}"))
prediction_triplexfpp_genocode_all.rename(columns={'string_5':'transcript_sym','string_6':'gene_sym_y'},inplace =True)
triplexfpp_genocode_all_filtered   =prediction_triplexfpp_genocode_all[["gene_sym_y","transcript_sym","triplex_helices","score"]]

In [128]:
tripleFPP_triplex_genecode   = triplexfpp_genocode_all_filtered.query('triplex_helices == "Triplex"')
tripleFPP_triplex_genecode.sort_values('score',ascending=False)

,gene_sym_y,transcript_sym,triplex_helices,score
12072,CR381653.1,CR381653.1-204,Triplex,1.151553
12070,CR381653.1,CR381653.1-202,Triplex,1.148390
7674,MIR100HG,MIR100HG-220,Triplex,1.129234
12028,AL355803.1,AL355803.1-201,Triplex,1.128435
10483,AC008105.3,AC008105.3-204,Triplex,1.111797
...,...,...,...,...
2351,ZMYND10-AS1,ZMYND10-AS1-201,Triplex,0.492086
3912,AC025188.1,AC025188.1-201,Triplex,0.491564
4070,LINC01184,LINC01184-205,Triplex,0.491360
4793,TAB2-AS1,TAB2-AS1-202,Triplex,0.490859


In [142]:
tripleFPP_triplex_genecode.query('gene_sym_y == "PVT1"')

,gene_sym_y,transcript_sym,triplex_helices,score
6061,PVT1,PVT1-346,Triplex,0.688675
6062,PVT1,PVT1-316,Triplex,0.647226
6063,PVT1,PVT1-234,Triplex,0.747205
6064,PVT1,PVT1-315,Triplex,0.654007
6065,PVT1,PVT1-304,Triplex,0.670471
...,...,...,...,...
6161,PVT1,PVT1-277,Triplex,0.597450
6162,PVT1,PVT1-361,Triplex,0.602169
6163,PVT1,PVT1-211,Triplex,0.688601
6164,PVT1,PVT1-375,Triplex,0.656961


In [130]:
tripleFPP_triplex_genecode.gene_sym_y.drop_duplicates().shape

(363,)

In [101]:
DE_brca_lncRNAs = pd.read_csv('/home/alva/Documents/lncRNAs_project_2020/Analysis_2021/Tcga_R/GDCdata/TCGA-BRCA/harmonized/Transcriptome_Profiling/Gene_Expression_Quantification/r_data/lncRNA_0.58_BRCA_DE',header=0,sep='\t')
positve_lncRNAs_triplexFPP = pd.read_csv('/home/alva/Documents/lncRNAs_project_2020/scripts_2021/TriplexFPP_data/data/triplexlncRNA_pos_lncRNAs',names=['gene_sym_y'],sep='\t')

In [136]:
dfs      = [DE_brca_lncRNAs,positve_lncRNAs_triplexFPP,tripleFPP_triplex_genecode]
df_final = reduce(lambda left,right: pd.merge(left,right,on='gene_sym_y'), dfs)
df_final.gene_sym_x.drop_duplicates()

0      SLX1A-SULT1A3
5             HOTAIR
10              PVT1
115        SSTR5-AS1
116       AL121832.2
117       AC069281.2
118          MIR22HG
132       MIRLET7BHG
135             MEG3
184       AL162586.1
185         MIR100HG
273           DIO3OS
275         TBX2-AS1
276           PICSAR
Name: gene_sym_x, dtype: object

In [ ]:
# The common DE lncRNAs between 15 cancer types

In [2]:
path         = '/home/alva/Documents/lncRNAs_project_2020/Analysis_2021/Tcga_R/GDCdata/TCGA-BRCA/harmonized/Transcriptome_Profiling/Gene_Expression_Quantification/r_data_new/de' 
files        = os.listdir(path)
results      = [os.path.join(path,i) for i in files if i.startswith('lncRNA_0.58_TCGA')]

In [54]:
dfs =list()
final_df = pd.DataFrame()
for lncde in results:
    name = os.path.basename(os.path.normpath(lncde))
    df   = pd.read_csv(lncde, sep='\t',header=0,skipinitialspace=True,usecols=["gene_id","external_gene_name"])
    df.drop_duplicates(inplace=True)
    df['filename'] = name
    df_gene = df[["external_gene_name","filename"]]
    #print("Reading file: {}".format(os.path.basename(os.path.normpath(lncde))))
    dfs.append(df_gene)
    frame=pd.concat(dfs,ignore_index=True,axis=0)
frame['filename']=frame['filename'].apply(lambda x:x.split('_')[2])

,external_gene_name,filename
0,AC016026.1,lncRNA_0.58_TCGA-HNSC_DE
1,IGF2-AS,lncRNA_0.58_TCGA-HNSC_DE
2,C22orf24,lncRNA_0.58_TCGA-HNSC_DE
3,AL139352.1,lncRNA_0.58_TCGA-HNSC_DE
4,LINC00525,lncRNA_0.58_TCGA-HNSC_DE
...,...,...
2376,AP003500.1,lncRNA_0.58_TCGA-HNSC_DE
2377,SAMD12-AS1,lncRNA_0.58_TCGA-HNSC_DE
2378,AP000851.2,lncRNA_0.58_TCGA-HNSC_DE
2379,AL732314.10,lncRNA_0.58_TCGA-HNSC_DE


In [29]:
de_lncRNA_accross_cancers      =frame.groupby(['external_gene_name']).size()
de_lncRNA_accross_cancers_count=de_lncRNA_accross_cancers.to_frame(name = 'size').reset_index()

In [196]:
# grouped_multiple.groupby([ 'external_gene_name' ]).agg({'count':'sum','filename':'count'})
grouped_multiple = frame.groupby(['external_gene_name','filename']).agg({'external_gene_name':'count'})
grouped_multiple.rename(columns={'external_gene_name':'count'},inplace =True)
grouped_multiple.reset_index(inplace =True)
grouped_multiple['count']= grouped_multiple.groupby(['external_gene_name'])['filename'].transform('count')
lncRNAs_accross_cancer   = grouped_multiple.query('count >=8')

NEK6


/home/alva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/alva/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4446: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [204]:
lncRNAs_accross_cancer['gene_sym'].drop_duplicates()



0          A1BG-AS1
9           A2M-AS1
91       AC002066.1
115      AC002116.2
125      AC002128.1
            ...    
34370    ZNF503-AS1
34398    ZNF582-AS1
34409    ZNF667-AS1
34427    ZNF710-AS1
34436    ZNF775-AS1
Name: gene_sym, Length: 1109, dtype: object

In [12]:
df_final = reduce(lambda left,right: pd.merge(left,right,on='gene_id'), dff)
df_final

,gene_id,external_gene_name_x,logFC_x,AveExpr_x,P.Value_x,adj_pval_x,TCGA-T3-A92M-01A-31R-A39I-07,TCGA-MT-A67F-01A-11R-A30B-07,TCGA-CR-7385-01A-11R-2016-07,TCGA-DQ-7588-01A-11R-2081-07,...,TCGA-56-8309-11A-01R-2296-07,TCGA-90-7964-01A-21R-2187-07,TCGA-18-3408-01A-01R-0980-07,TCGA-18-4721-01A-01R-1443-07,TCGA-22-5478-01A-01R-1635-07,chr_y,start_y,end_y,gene_sym_y,filename_y
0,ENSG00000163915,IGF2BP2-AS1,2.274179,-2.891034,1.040146e-10,7.670303e-10,0.379185,-1.724221,-3.433873,-1.686740,...,-6.275896,-3.739804,-1.652933,-3.918905,-3.894190,3,185712528,185729787,IGF2BP2-AS1,lncRNA_0.58_TCGA-LUSC_DE
1,ENSG00000167912,AC090152.1,0.984436,-0.570289,9.843145e-05,2.743424e-04,0.447491,0.107656,-0.626518,-2.457258,...,0.544282,-1.237304,-0.147042,0.971866,-1.799032,8,59119040,59123478,AC090152.1,lncRNA_0.58_TCGA-LUSC_DE
2,ENSG00000172965,MIR4435-2HG,1.032232,4.053967,1.057449e-10,7.783478e-10,4.552506,5.326716,3.902156,5.844642,...,3.276773,4.986982,5.582411,5.178249,4.825355,2,111036776,111523376,MIR4435-2HG,lncRNA_0.58_TCGA-LUSC_DE
3,ENSG00000176912,TYMSOS,0.872947,-0.326888,9.978939e-05,2.778598e-04,-1.104494,-0.485434,0.062553,1.162639,...,-2.816465,0.032786,-0.067971,0.939076,0.831254,18,623743,658340,TYMSOS,lncRNA_0.58_TCGA-LUSC_DE
4,ENSG00000179406,LINC00174,1.481817,2.848914,4.377788e-10,2.944432e-09,3.421193,2.809342,2.744614,4.364422,...,3.043776,3.707739,2.246243,0.905523,3.211481,7,66376044,66493566,LINC00174,lncRNA_0.58_TCGA-LUSC_DE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,ENSG00000276853,AC026124.2,-0.836269,-2.983893,8.478026e-06,2.895799e-05,-2.739210,-0.586717,-2.540788,-3.085289,...,-2.575457,-4.587801,-2.596349,-2.044436,-3.266158,12,65171262,65171917,AC026124.2,lncRNA_0.58_TCGA-LUSC_DE
431,ENSG00000276855,AC015922.3,-1.217979,-0.151390,2.582322e-10,1.791180e-09,1.256037,-1.238794,-0.749374,-2.637830,...,2.585190,1.887933,1.591954,-0.294414,2.873995,17,15789016,15789705,AC015922.3,lncRNA_0.58_TCGA-LUSC_DE
432,ENSG00000277954,AC092376.2,-0.583065,-1.660229,6.112240e-03,1.188627e-02,-0.878613,-4.783115,-1.993300,-3.374796,...,-1.417915,-1.355140,-1.541902,0.562222,1.017066,16,79202624,79206739,AC092376.2,lncRNA_0.58_TCGA-LUSC_DE
433,ENSG00000278376,AP004609.3,-1.145908,-0.945578,2.335794e-12,2.154131e-11,-0.612098,-1.413881,-0.626518,-2.844281,...,0.353460,0.838464,-0.463899,0.050721,-0.115216,11,118791254,118793137,AP004609.3,lncRNA_0.58_TCGA-LUSC_DE


In [ ]:
dfs      = [DE_brca_lncRNAs,positve_lncRNAs_triplexFPP,tripleFPP_triplex_genecode]
df_final = reduce(lambda left,right: pd.merge(left,right,on='gene_sym_y'), dfs)
df_final.gene_sym_x.drop_duplicates()

In [143]:
de_brca_positive = pd.merge(DE_brca_lncRNAs,positve_lncRNAs_triplexFPP,on='gene_sym_y')
de_brca_positive_triplexgencode = pd.merge(DE_brca_lncRNAs,tripleFPP_triplex_genecode,on='gene_sym_y')
de_brca_positive_triplexgencode.to_csv('/home/alva/Documents/lncRNAs_project_2020/Analysis_2021/Tcga_R/GDCdata/TCGA-BRCA/harmonized/Transcriptome_Profiling/Gene_Expression_Quantification/r_data/de_lncRNAs_with_triplex/gencode_triplex_predictions_brca_de',index=False)
